In [4]:
# Import packages
import pandas as pd
import pickle

In [5]:
# Import loyalty_score_data
loyalty_scores = pd.read_excel('../../Data/grocery_database.xlsx', sheet_name='loyalty_scores')
loyalty_scores.head(1)

,customer_id,customer_loyalty_score
0,104,0.587


In [6]:
# Import customer_detail_data
customer_details = pd.read_excel('../../Data/grocery_database.xlsx', sheet_name='customer_details')
customer_details.head(1)

,customer_id,distance_from_store,gender,credit_score
0,74,3.38,F,0.59


In [7]:
# Import transaction_data
transactions = pd.read_excel('../../Data/grocery_database.xlsx', sheet_name='transactions')
transactions.head(1)

,customer_id,transaction_date,transaction_id,product_area_id,num_items,sales_cost
0,1,2020-04-10,435657533999,3,7,19.16


In [8]:
# Merge data from customer_detail and loyalty score to build initial dataframe
df0 = pd.merge(customer_details,
              loyalty_scores,
              how='left',
              on='customer_id'
              )
print(df0.shape)
df0.head(3)

(870, 5)


,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score
0,74,3.38,F,0.59,0.263
1,524,4.76,F,0.52,0.298
2,607,4.45,F,0.49,0.337


In [9]:
# Aggregate transaction data so it can merge into df0

sales_summary_df = (transactions
                    .groupby('customer_id')
                    .agg({'sales_cost':'sum',
                          'num_items':'sum',
                          'transaction_id':'count',
                          'product_area_id':'nunique',
                          }
                         )
                    .reset_index()
                    )
sales_summary_df.columns = ['customer_id','total_sales','total_items', 'transaction_count', 'product_area_count']
print(sales_summary_df.shape)
sales_summary_df.head(3)

(870, 5)


,customer_id,total_sales,total_items,transaction_count,product_area_count
0,1,3980.49,424,51,5
1,2,2056.91,213,52,5
2,3,324.22,65,12,4


In [10]:
# Add a column for the average basket value that might add insight to the model
sales_summary_df['average_basket_value'] = sales_summary_df['total_sales']/sales_summary_df["transaction_count"]
sales_summary_df.head(1)

,customer_id,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,1,3980.49,424,51,5,78.048824


In [ ]:
# Merge the aggregate dataframe with initial dataframe
df1 = pd.merge(df0, sales_summary_df, how='inner', on='customer_id')
df1.head(3)

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224


In [15]:
# Create dataframe with labels for model training
regression_data = df1.loc[df1['customer_loyalty_score'].notna()]
# Create dataframe without labels for model to be used on
regression_scoring = df1.loc[df1['customer_loyalty_score'].isna()]
regression_scoring.drop(['customer_loyalty_score'],axis=1)
regression_scoring.head(1)

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
6,1,4.78,F,0.66,NaN,3980.49,424,51,5,78.048824


In [17]:
pickle.dump(regression_data, open('../../Data/Processed/regression_data.p','wb'))

In [18]:
pickle.dump(regression_scoring, open('../../Data/Processed/regression_scoring.p','wb'))